In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%ls 

gdrive/  sample_data/


In [4]:
%cd gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP/

/content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP


In [5]:
df = pd.read_csv('preprossData.csv')

In [6]:
df.head()

,Unnamed: 0,title,ingred,instructions
0,0,Worlds Best Mac and Cheese,"penne|beechers flagship cheese sauce|cheddar, ...",preheat the oven to 350 f. butter or oil an 8-...
1,1,Dilly Macaroni Salad Recipe,c. eow macaroni|c. cubed american cheese|c. sl...,cook macaroni according to package directions;...
2,2,Gazpacho,"maes, quartered|kosher salt|red onion, cut in ...",add the tomatoes to a food processor with a pi...
3,3,Crunchy Onion Potato Bake,"milk|water|butter|mashed potaes, box, homesty...",preheat oven to 350 degrees fahrenheit.spray p...
4,4,Cool 'n Easy Creamy Watermelon Pie,package watermelon gelatin|boiling water|packa...,dissolve jello in boiling water.allow to cool ...


In [7]:
test = df.iloc[330]
test

Unnamed: 0                                                    330
title                                     Fast Hammy Grits Recipe
ingred          water|whole milk|tablespoon unsalted butter|ko...
instructions    combine the water, milk, butter, and measured ...
Name: 330, dtype: object

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.9 MB/s eta 0:00:00


In [14]:
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
# import pytorch_lighting as pl
from sklearn.model_selection import train_test_split
import textwrap

In [15]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [21]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [39]:
sample_ip = tokenizer(test['ingred'])
input_id = sample_ip['input_ids']


In [40]:
tokenizer.decode(sample_ip['input_ids'])

'water|whole milk|tablespoon unsalted butter|kosher salt, plus more as needed|freshly ground black pepper, plus more as needed|s vegetable oil|small-dice, fully cooked ham|white or yellow grits|sharp cheddar cheese|medium scallion|hot sauce, such as cholula or tabasco'

In [37]:
preds = [tokenizer.decode(input_id, skip_special_token = True, clean_up_tokenization_spaces=True) for input_id in sample_ip['input_ids'] ]

In [38]:
" ".join(preds)

'water | wh ole  milk | t ables poon  uns alted  butter | k osher  salt ,  plus  more  as  needed | fresh ly  ground  black  pepper ,  plus  more  as  needed | s  vegetable  oil | small - d ice ,  fully  cooked  ham | white  or  yellow  grit s | sharp  c heddar  cheese | medium  sc allion | hot  sauce ,  such  as  ch ol ula  or  tab as co Fast  Ham my  G rit s  Recipe'

In [43]:
encoding_ip = tokenizer(
    test['ingred'],
    return_attention_mask = True,
    return_tensors = "pt"
)

In [44]:
encoding_ip.keys()

dict_keys(['input_ids', 'attention_mask'])

In [45]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [46]:
tokenizer.bos_token_id, tokenizer.eos_token_id

(50256, 50256)

In [47]:
tokenizer.decode(encoding_ip['input_ids'].squeeze())

'water|whole milk|tablespoon unsalted butter|kosher salt, plus more as needed|freshly ground black pepper, plus more as needed|s vegetable oil|small-dice, fully cooked ham|white or yellow grits|sharp cheddar cheese|medium scallion|hot sauce, such as cholula or tabasco'

In [52]:
encoding_op = tokenizer(
    test['instructions'],
    return_attention_mask = True,
    truncation = False,
    add_special_tokens = True,
    return_tensors = "pt"
)

In [53]:
tokenizer.decode(encoding_op['input_ids'].squeeze())

'combine the water, milk, butter, and measured salt and pepper in a small saucepan and bring to a simmer over medium-high heat.meanwhile, heat the oil in a medium frying pan over medium heat until shimmering.add the ham and cook, stirring occasionally, until browned all over, about 5 to 6 minutes.remove the pan from the heat and set it aside.whisk the grits into the simmering milk mixture and bring to a boil.continue to boil without stirring for 1 minute.cover with a tightfitting lid, remove the pan from the heat, and let sit until the grits are tender, about 7 minutes.meanwhile, shred the cheese on the large holes of a box grater (you should have about 3/4 cup).thinly slice the white and light green parts of the scallion; set aside.when the grits are ready, uncover and stir them to incorporate all of the liquid; stir until the grits are smooth.sprinkle with two-thirds of the cheese and stir until combined and melted.taste and season with salt and pepper as needed.divide the grits betw

In [58]:
class RecipeDataset(Dataset):
  def __init__(self, data: pd.DataFrame, tokenizer: GPT2Tokenizer):
      self.tokenizer = tokenizer
      self.data = data
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    encoding_ip = tokenizer(data_row['ingred'], return_attention_mask = True, return_tensors = "pt")
    encoding_op = tokenizer(data_row['instructions'], return_attention_mask = True, return_tensors = "pt")
    return {
        'ingred': data_row['ingred'],
        'instructions': data_row['instructions'],
        'input_ids': encoding_ip['input_ids'].flatten(),
        'output_ids': encoding_op['input_ids'].flatten(),
        'attention_mask_ip': encoding_ip['attention_mask'].flatten(),
        'attention_mask_op': encoding_op['attention_mask'].flatten()
    }


In [59]:
sample_dataset = RecipeDataset(df, tokenizer)

In [69]:
count = 0
for i in sample_dataset:
  print(i['ingred'])
  print(i['input_ids'])
  print(len(i['output_ids']))
  break


penne|beechers flagship cheese sauce|cheddar, grated|gruyere cheese, grated|chipotle chili powder|unsalted butter|all-purpose flour|milk|semihard cheese, grated|semisoft cheese, grated|kosher salt|chipotle chili powder|garlic powder
tensor([ 3617,   710,    91,  1350,   721,  7084, 21336,  9891, 10746,    91,
         1740, 27455,    11,  1036,   515,    91, 48929,    88,   567,  9891,
           11,  1036,   515,    91, 35902, 23556, 32728, 11913,    91, 13271,
        29590,  9215,    91,   439,    12, 29983, 10601,    91, 25433,    74,
           91, 43616,  4449,   446,  9891,    11,  1036,   515,    91,   325,
        25413, 11205,  9891,    11,  1036,   515,    91,    74, 38321,  8268,
           91, 35902, 23556, 32728, 11913,    91,  4563,   677, 11913])
336


In [68]:
df.iloc[8]['ingred']

', weight light fat free vanilla yogurt|fresh sliced strawberries|low-fat granola'

In [77]:
train_df, test_df = train_test_split(df, test_size = 0.05)

In [79]:
train_df.shape, test_df.shape

((978234, 4), (51486, 4))

In [80]:
train_dataset = RecipeDataset(train_df, tokenizer)
test_dataset = RecipeDataset(test_df, tokenizer)

In [83]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers = 2)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers = 2)